# **Instalacion de Kaggle**

In [ ]:
pip install kaggle

Se agrega el archivo kaggle.json a los files

In [16]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Cargar imagenes

In [ ]:
! kaggle datasets download -d puneet6060/intel-image-classification

In [ ]:
! unzip intel-image-classification.zip

Acomodar las imagenes en arreglos

In [ ]:
import cv2
import glob
import numpy as np
class_names = [ 'buildings','forest', 'glacier', 'mountain', 'sea', 'street']
test_images = []
test_labels = []
i_i = -1
i_f = 0
size = 150
for i in class_names:
  i_i = i_i + 1
  files = glob.glob ('seg_test/seg_test/' + i + '/*')
  for f in files:
    i_f = i_f + 1
    image = cv2.imread (f)
    image = np.resize(image,(size, size, 3))
    test_images.append(image)
  s = (i_f,1)
  i_f = 0
  label = []
  label = np.ones(s)*i_i
  for n in label:
    test_labels.append([label])
  i_i = 0

print(np.array(test_images).shape)
print(np.array(test_labels).shape)

train_images = []
train_labels = []
i_f = 0
i_i = -1
size = 150
for i in class_names:
  i_i = i_i + 1
  files = glob.glob ('seg_train/seg_train/' + i + '/*')
  for f in files:
    i_f = i_f + 1
    image = cv2.imread (f)
    image = np.resize(image,(size, size, 3))
    train_images.append(image)
  s = (i_f,1)
  i_f = 0
  label = []
  label = np.ones(s)*i_i
  for n in label:
    train_labels.append([label])
  i_i = 0

print(np.array(train_images).shape)
print(np.array(train_labels).shape)

Pre-procese el set de datos

In [ ]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Librerias de ayuda
import numpy as np
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()


In [40]:
train_images= np.divide(train_images, 255.0)
test_images= np.divide(test_images, 255.0)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

Configurar las capas

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Entrenar modelo

In [ ]:
model.fit(train_images, train_labels, epochs=10)

Evaluar exactitud

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Hacer predicciones

In [ ]:
predictions = model.predict(test_images)
predictions[0]
np.argmax(predictions[0])
test_labels[0]

Graficar

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

predictions_single = model.predict(img)

print(predictions_single)

plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

np.argmax(predictions_single[0])